<a href="https://colab.research.google.com/github/mostafa-ja/mal_adv4/blob/main/adverserial_attacks_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,balanced_accuracy_score
from scipy import sparse
import gdown
import numpy as np
import matplotlib.pyplot as plt
import pickle
import torch.nn.functional as F
import random

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
class MalwareDetectionModel(nn.Module):
    def __init__(self, input_size=10000, hidden_1_size=200, hidden_2_size=200, num_labels=2, dropout_prob=0.6):
        super(MalwareDetectionModel, self).__init__()

        self.input_size = input_size
        self.hidden_1_size = hidden_1_size
        self.hidden_2_size = hidden_2_size
        self.num_labels = num_labels
        self.dropout_prob = dropout_prob

        self.fc1 = nn.Linear(input_size, hidden_1_size)
        self.relu1 = nn.ReLU()
        #self.dropout1 = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(hidden_1_size, hidden_2_size)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_prob)
        self.fc3 = nn.Linear(hidden_2_size, num_labels)
        #self.log_softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        #x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        #x = self.log_softmax(x)
        return x


In [ ]:
def adjust_learning_rate(optimizer, epoch, lr_step=(20,30,40), lr_decay_ratio=0.2):
    """Adjust the learning rate based on the epoch number."""
    if epoch == 0:
        optimizer.param_groups[0]['lr'] /= 8
    elif epoch in [1, 2, 3]:  # in step five , we finish warm up ,and start normal learning rate
        optimizer.param_groups[0]['lr'] *= 2
    if epoch in lr_step: # in these steps , we are geting close to optimal point so we need to have shorter step
        optimizer.param_groups[0]['lr'] *= lr_decay_ratio
    return optimizer


In [ ]:
def adversarial_training(model, train_loader, val_loader, attack, adv_epochs=50, lr=0.001, weight_decay=0., device=device, verbose=True, **kwargs):

    # Assuming positive class (malware) is label 1
    #class_weights = torch.tensor([0.11, 0.89]).to(device)  # Adjust the weights based on the class distribution, higher weight for positive class

    # Define Loss Function and Optimizer
    #criterion = nn.CrossEntropyLoss(weight=class_weights)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    total_time = 0.
    nbatches = len(train_loader)
    best_acc_val = 0.
    acc_val_adv_be = 0.
    best_epoch = 0

    for epoch in range(adv_epochs):
        epoch_losses = []
        epoch_accuracies = []
        optimizer = adjust_learning_rate(optimizer, epoch)

        for idx_batch, (x_batch, y_batch) in enumerate(train_loader):
            x_batch, y_batch = x_batch.to(torch.float32).to(device), y_batch.to(device)
            batch_size = x_batch.shape[0]

            # Separate malicious and benign samples
            mal_x_batch, ben_x_batch = x_batch[y_batch.squeeze() == 1], x_batch[y_batch.squeeze() == 0]
            mal_y_batch, ben_y_batch = y_batch[y_batch.squeeze() == 1], y_batch[y_batch.squeeze() == 0]

            # Generate adversarial examples
            model.eval()
            pertb_mal_x = attack(mal_x_batch, mal_y_batch, model, **kwargs)
            x_batch = torch.cat([ben_x_batch, pertb_mal_x], dim=0)
            y_batch = torch.cat([ben_y_batch, mal_y_batch])
            model.train()

            # Forward pass and backward pass
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss_train = criterion(outputs, y_batch.view(-1).long())
            loss_train.backward()
            optimizer.step()

            # Calculate metrics
            epoch_losses.append(loss_train.item())
            predicted = outputs.argmax(1).unsqueeze(1)
            acc_train = (predicted == y_batch).sum().item() / len(y_batch)
            epoch_accuracies.append(acc_train)

            # Print batch level information
            #if verbose:
                #print(f'Mini batch: {idx_batch + 1}/{nbatches} | Epoch: {epoch + 1}/{adv_epochs} | Batch Loss: {loss_train.item():.4f} | Batch Accuracy: {acc_train * 100:.2f}%')

        # Calculate epoch level metrics
        mean_loss = np.mean(epoch_losses)
        mean_accuracy = np.mean(epoch_accuracies) * 100

        # Print epoch level information
        if verbose:
            print(f'Epoch: {epoch+1}/{adv_epochs} | Training loss (epoch level): {mean_loss:.4f} | Train accuracy: {mean_accuracy:.2f}%')

        # Evaluation on validation set
        model.eval()
        cor_val = 0
        cor_ad_val = 0
        n_samples = 0
        n_ad_samples = 0
        for x_val, y_val in val_loader:
            x_val, y_val = x_val.to(torch.float32).to(device), y_val.to(device)
            n_samples += len(x_val)
            outputs = model(x_val)
            predicted = outputs.argmax(1).unsqueeze(1)
            cor_val += (predicted == y_val).sum().item()

            # Generate adversarial examples for validation set
            mal_x_batch, mal_y_batch = x_val[y_val.squeeze() == 1], y_val[y_val.squeeze() == 1]
            n_ad_samples += len(mal_x_batch)
            pertb_mal_x = attack(mal_x_batch, mal_y_batch, model, **kwargs)
            outputs = model(pertb_mal_x)
            y_pred = outputs.argmax(1)
            cor_ad_val += (y_pred == 1.).sum().item()

        # Calculate validation accuracy
        assert n_ad_samples > 0
        avg_acc_val = (cor_val / n_samples)
        avg_acc_ad_val = (cor_ad_val / n_ad_samples)
        acc_all = (avg_acc_val + avg_acc_ad_val) / 2.

        # Update best validation accuracy
        if acc_all >= best_acc_val:
            best_acc_val = acc_all
            acc_val_adv_be = avg_acc_ad_val
            best_epoch = epoch + 1
            torch.save(model.state_dict(), 'model_AT_rFGSM.pth')

        # Print validation results
        if verbose:
            print(f"\tVal accuracy(without attack) {(avg_acc_val) * 100:.4}% and accuracy(with attack) {(avg_acc_ad_val) * 100:.4}% under attack and overall accuracy {acc_all * 100:.4}%.")
            print(f"\tModel select at epoch {best_epoch} with validation accuracy {best_acc_val * 100:.4}% and accuracy {acc_val_adv_be * 100:.4}% under attack.")


In [ ]:
def adv_predict(test_loader, model, attack, device, **kwargs):

    if attack == mimicry:
      # Pre-select benign samples
      benign_samples = []
      for x_batch, y_batch in test_loader:
        benign_samples.append(x_batch[y_batch.squeeze() == 0])

      ben_x = torch.cat(benign_samples, dim=0).to(device)
      del benign_samples

    model.eval()
    n_samples = 0
    cor_test = 0
    cor_ad_test = 0

    with torch.no_grad():
        for x_test, y_test in test_loader:
            x_test, y_test = x_test.to(torch.float32).to(device), y_test.to(device)
            #outputs = model(x_test)
            #predicted = outputs.argmax(1).unsqueeze(1)
            #acc_test = (predicted == y_test).sum().item() / len(y_test)
            #avg_acc_test.append(acc_test)

            mal_x_batch, mal_y_batch = x_test[y_test.squeeze() == 1], y_test[y_test.squeeze() == 1]
            n_samples += len(mal_y_batch)

            outputs = model(mal_x_batch)
            predicted = outputs.argmax(1)
            cor_test += (predicted == 1).sum().item()

            # Generate adversarial examples for test set
            if attack == mimicry:
                pertb_mal_x = mimicry(ben_x, mal_x_batch, model, **kwargs)
            else :
                with torch.enable_grad():
                    pertb_mal_x = attack(mal_x_batch, mal_y_batch, model, **kwargs)

            outputs = model(pertb_mal_x)
            y_pred = outputs.argmax(1)
            cor_ad_test += (y_pred == 1).sum().item()

    # Calculate test accuracy
    print(f"Accuracy of just malwares (without attack): {(cor_test / n_samples) * 100:.4}% | Under attack: {(cor_ad_test / n_samples) * 100:.4}%.")
    if attack == mimicry:
      del ben_x


In [ ]:
def round_x(x, round_threshold=0.5):
    """
    Rounds x by thresholding it according to round_threshold.
    :param x: input tensor
    :param round_threshold: threshold parameter
    :return: a tensor of 0s and 1s
    """
    return (x >= round_threshold).float()

def get_x0(x, initial_rounding_threshold=0.5, is_sample=False):
    """
    Helper function to randomly initialize the inner maximizer algorithm.
    Randomizes the input tensor while preserving its functionality.
    :param x: input tensor
    :param rounding_threshold: threshold for rounding
    :param is_sample: flag to sample randomly from feasible area
    :return: randomly sampled feasible version of x
    """
    if is_sample:
        rand_x = round_x(torch.rand(x.size()), initial_rounding_threshold=initial_rounding_threshold)
        return (rand_x.byte() | x.byte()).float()
    else:
        return x

def or_float_tensors(x_1, x_2):
    """
    ORs two float tensors by converting them to byte and back.
    :param x_1: tensor one
    :param x_2: tensor two
    :return: float tensor of 0s and 1s
    """
    return (x_1.byte() | x_2.byte()).float()


def xor_float_tensors(x_1, x_2):
    """
    XORs two float tensors by converting them to byte and back
    Note that byte() takes the first 8 bit after the decimal point of the float
    e.g., 0.0 ==> 0
          0.1 ==> 0
          1.1 ==> 1
        255.1 ==> 255
        256.1 ==> 0
    Subsequently the purpose of this function is to map 1s float tensors to 1
    and those of 0s to 0. I.e., it is meant to be used on tensors of 0s and 1s.

    :param x_1: tensor one
    :param x_2: tensor two
    :return: float tensor of 0s and 1s.
    """
    return (x_1.byte() ^ x_2.byte()).float()

def get_loss(x,y,model):
    criterion = nn.CrossEntropyLoss(reduction='none')
    outputs = model(x)
    loss = criterion(outputs, y.view(-1).long())
    _, predicted = torch.topk(outputs, k=1)
    done = (predicted != y).squeeze()

    return loss, done



In [ ]:
def model_evaluation(model, test_loader, device):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    true_labels = []

    with torch.no_grad():
        for X_batch, labels_batch in test_loader:
            X_batch, labels_batch = X_batch.to(torch.float32).to(device), labels_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.topk(outputs, k=1)
            predictions.extend(predicted.tolist())
            true_labels.extend(labels_batch.tolist())

    # Convert predictions and true labels to numpy arrays
    predictions = np.array(predictions)
    true_labels = np.array(true_labels)

    # Calculate and print test accuracy
    accuracy = accuracy_score(true_labels, predictions)
    balanced_acc = balanced_accuracy_score(true_labels, predictions)
    print(f'Test Accuracy: {accuracy:.4f}')
    print(f'Test balanced Accuracy: {balanced_acc:.4f}')

    # Calculate and print precision, recall, and F1-score
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)

    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1-score: {f1:.4f}')

    # Calculate and print true positives, true negatives, false positives, and false negatives
    TP = ((predictions == 1) & (true_labels == 1)).sum()
    TN = ((predictions == 0) & (true_labels == 0)).sum()
    FP = ((predictions == 1) & (true_labels == 0)).sum()
    FN = ((predictions == 0) & (true_labels == 1)).sum()

    print(f'True Positives (TP): {TP}')
    print(f'True Negatives (TN): {TN}')
    print(f'False Positives (FP): {FP}')
    print(f'False Negatives (FN): {FN}')

    # Calculate and print False Negative Rate (FNR) and False Positive Rate (FPR)
    FNR = (FN / (FN + TP)) * 100
    FPR = (FP / (FP + TN)) * 100

    print(f'False Negative Rate (FNR): {FNR:.4f}')
    print(f'False Positive Rate (FPR): {FPR:.4f}')

In [ ]:
def dfgsm_k(x, y, model, k=25, epsilon=0.02, alpha=1., initial_rounding_threshold=0.5, round_threshold=0.5, random=False, is_report_loss_diff=True, is_sample=False, device="cpu"):
    """
    FGSM^k with deterministic rounding
    :param y: ground truth labels
    :param x: feature vector
    :param model: neural network model
    :param k: number of steps
    :param epsilon: update value in each direction
    :param alpha: hyperparameter for controlling the portionate of rounding
    :param initial_rounding_threshold: threshold parameter for rounding the initial x_next
    :param round_threshold: threshold parameter for rounding
    :param is_report_loss_diff: flag to report loss difference
    :param is_sample: flag to sample randomly from the feasible area
    :return: the adversarial version of x according to dfgsm_k (tensor)
    """
    model.eval()

    # Compute natural loss
    criterion = nn.CrossEntropyLoss(reduction='none')
    loss_natural = criterion(model(x), y.view(-1).long())

    # Initialize starting point
    x_next = x.clone()
    x_next = get_x0(x_next, initial_rounding_threshold, is_sample)

    # Multi-step
    for t in range(k):
        # Forward pass
        x_var = x_next.clone().detach().requires_grad_(True)
        y_model = model(x_var)
        loss = criterion(y_model, y.view(-1).long())

        # Compute gradient
        grad_vars = torch.autograd.grad(loss.mean(), x_var)
        # Find the next sample
        x_next = x_next + epsilon * torch.sign(grad_vars[0].data)

        # Projection
        x_next = torch.clamp(x_next, min=0., max=1.)

    # Rounding step
    if random:
       round_threshold = (torch.rand(x_next.size()) * alpha).to(device=x.device)
    x_next = round_x(x_next, round_threshold=round_threshold)

    # Feasible projection
    x_next = or_float_tensors(x_next, x)

    # Compute adversarial loss
    loss_adv = criterion(model(x_next), y.view(-1).long()).data

    if is_report_loss_diff:
        #print(f"Natural loss: {loss_natural.mean():.4f}, Adversarial loss: {loss_adv.mean():.4f}, Difference: {(loss_adv.mean() - loss_natural.mean()):.4f}")
        outputs = model(x_next)
        _, predicted = torch.topk(outputs, k=1)
        done = (predicted != y).squeeze()
        print(f"dFGSM: attack effectiveness {done.sum().item() / x.size()[0] * 100:.3f}%.")

    # Replace with natural if adversarial loss is higher
    replace_flag = (loss_adv < loss_natural).squeeze()
    x_next[replace_flag] = x[replace_flag]

    return x_next


In [ ]:
def bga_k(x, y, model, k=25, alpha=1., is_report_loss_diff=True, use_sample=False):
    """
    Multi-step bit gradient ascent
    :param x: feature vector
    :param y: ground truth labels
    :param model: neural network model
    :param k: number of steps
    :param alpha: hyperparameter for controlling updates
    :param is_report_loss_diff: flag to report loss difference
    :param use_sample: flag to sample randomly from the feasible area
    :return: the adversarial version of x according to bga_k (tensor)
    """
    model.eval()

    # Compute natural loss
    criterion = nn.CrossEntropyLoss(reduction='none')
    loss_natural = criterion(model(x), y.view(-1).long())

    # Initialize worst loss and corresponding adversarial samples
    loss_worst = loss_natural.clone()
    x_worst = x.clone()

    # Book-keeping
    sqrt_m = (torch.sqrt(torch.tensor([x.size()[1]], dtype=torch.float))).to(x.device)

    # Multi-step with gradients
    for t in range(k):
        if t == 0:
            # Initialize starting point
            x_next = get_x0(x, use_sample)
        else:
            # Compute gradient
            grad_vars = torch.autograd.grad(loss.mean(), x_var)
            grad_data = grad_vars[0].data

            # Compute the updates
            # torch.norm(grad_data, 2, 1), 2:the L2-norm , 1:the norm along dimension 1
            x_update = (sqrt_m * (1. - 2. * x_next) * grad_data >= (alpha * torch.norm(grad_data, 2, 1).unsqueeze(1))).float()

            # Find the next sample with projection to the feasible set
            x_next = xor_float_tensors(x_update, x_next)
            x_next = or_float_tensors(x_next, x)

        # Forward pass
        x_var = x_next.clone().detach().requires_grad_(True)
        y_model = model(x_var)
        loss = criterion(y_model, y.view(-1).long())

        # Update worst loss and adversarial samples
        replace_flag = (loss.data > loss_worst)
        loss_worst[replace_flag] = loss.data[replace_flag]
        x_worst[replace_flag] = x_next[replace_flag]

    if is_report_loss_diff:
        #print(f"Natural loss: {loss_natural.mean():.4f}, Adversarial loss: {loss_worst.mean():.4f}, Difference: {(loss_worst.mean() - loss_natural.mean()):.4f}")
        outputs = model(x_worst)
        _, predicted = torch.topk(outputs, k=1)
        done = (predicted != y).squeeze()
        print(f"bga_k: attack effectiveness {done.sum().item() / x.size()[0] * 100:.3f}%.")

    return x_worst


In [ ]:
def bca_k(x, y, model, k=25, is_report_loss_diff=True, use_sample=False):
    """
    Multi-step bit coordinate ascent
    :param use_sample:
    :param is_report_loss_diff:
    :param y:
    :param x: (tensor) feature vector
    :param model: nn model
    :param k: num of steps
    :return: the adversarial version of x according to bca_k (tensor)
    """

    model.eval()

    # Compute natural loss
    criterion = nn.CrossEntropyLoss(reduction='none')
    loss_natural = criterion(model(x), y.view(-1).long())

    # keeping worst loss
    loss_worst = loss_natural.clone()
    x_worst = x.clone()

    # multi-step with gradients
    loss = None
    x_var = None
    x_next = None
    for t in range(k):
        if t == 0:
            # initialize starting point
            x_next = get_x0(x, use_sample)
        else:
            # compute gradient
            grad_vars = torch.autograd.grad(loss.mean(), x_var)
            grad_data = grad_vars[0].data

            # compute the updates (can be made more efficient than this)
            #aug_grad = (1. - 2. * x_next) * grad_data #this line is wrong because the grad_data can be negative
            aug_grad = (x_next < 0.5) * grad_data # the correct version
            val, _ = torch.topk(aug_grad, 1)
            x_update = (aug_grad >= val.expand_as(aug_grad)).float()
            # find the next sample with projection to the feasible set
            x_next = xor_float_tensors(x_update, x_next)
            x_next = or_float_tensors(x_next, x)

        # forward pass
        x_var = x_next.clone().detach().requires_grad_(True)
        y_model = model(x_var)
        loss = criterion(y_model, y.view(-1).long())

        # update worst loss and adversarial samples
        replace_flag = (loss.data > loss_worst)
        loss_worst[replace_flag] = loss.data[replace_flag]
        x_worst[replace_flag] = x_next[replace_flag]

    if is_report_loss_diff:
        #print("Natural loss (%.4f) vs Adversarial loss (%.4f), Difference: (%.4f)" %(loss_natural.mean(), loss_worst.mean(), loss_worst.mean() - loss_natural.mean()))
        outputs = model(x_worst)
        _, predicted = torch.topk(outputs, k=1)
        done = (predicted != y).squeeze()
        print(f"bca_k: attack effectiveness {done.sum().item() / x.size()[0] * 100:.3f}%.")



    return x_worst

In [ ]:
def grosse_k(x, y, model, k=25, is_report_loss_diff=True, use_sample=False):
    """
    Multi-step bit coordinate ascent
    :param use_sample:
    :param is_report_loss_diff:
    :param y:
    :param x: (tensor) feature vector
    :param model: nn model
    :param k: num of steps
    :return: the adversarial version of x according to bca_k (tensor)
    """
    epsilon = 1e-10 # avoid gradient less than epsilon

    model.eval()

    # Compute natural loss
    criterion = nn.CrossEntropyLoss(reduction='none')
    loss_natural = criterion(model(x), y.view(-1).long())

    # keeping worst loss
    loss_worst = loss_natural.clone()
    x_worst = x.clone()

    # multi-step with gradients
    output = None
    x_var = None
    x_next = None
    for t in range(k):
        if t == 0:
            # initialize starting point
            x_next = get_x0(x, use_sample)
        else:
            # compute gradient
            # ouput.shape=([batch_size, 2]) because of 2 neoruns, so we just use the output of the first neorun(benign)
            grad_vars = torch.autograd.grad(output[:, 0].mean(), x_var)
            grad_data = grad_vars[0].data

            # compute the updates (can be made more efficient than this)
            #aug_grad = (1. - x_next) * grad_data
            aug_grad = (x_next < 0.5) * grad_data
            val, _ = torch.topk(aug_grad, 1)
            x_update = ((aug_grad >= val.expand_as(aug_grad)).float()) * (aug_grad > epsilon)

            # find the next sample with projection to the feasible set
            x_next = xor_float_tensors(x_update, x_next)
            x_next = or_float_tensors(x_next, x)

        # forward pass
        x_var = x_next.clone().detach().requires_grad_(True)
        output = model(x_var)
        loss = criterion(output, y.view(-1).long())

        # update worst loss and adversarial samples
        replace_flag = (loss.data > loss_worst)
        loss_worst[replace_flag] = loss.data[replace_flag]
        x_worst[replace_flag] = x_next[replace_flag]

    if is_report_loss_diff:
        #print("Natural loss (%.4f) vs Adversarial loss (%.4f), Difference: (%.4f)" %(loss_natural.mean(), loss_worst.mean(), loss_worst.mean() - loss_natural.mean()))
        outputs = model(x_worst)
        _, predicted = torch.topk(outputs, k=1)
        done = (predicted != y).squeeze()
        print(f"grosse_k: attack effectiveness {done.sum().item() / x.size()[0] * 100:.3f}%.")



    return x_worst

In [ ]:
def pgd(x, y, model, insertion_array, removal_array, k=25, step_length=0.02, norm='linf', initial_rounding_threshold=0.5, round_threshold=0.5, random=False, is_report_loss_diff=True, is_sample=False):
    """
    Projected Gradient Descent (PGD) adversarial attack.
    :param y: Ground truth labels
    :param x: Feature vector
    :param model: Neural network model
    :param k: Number of steps
    :param step_length: Step size for each iteration
    :param norm: Norm used for perturbation ('linf' or 'l2')
    :param initial_rounding_threshold: Threshold parameter for rounding the initial x_next
    :param round_threshold: Threshold parameter for rounding
    :param random: Flag to generate random thresholds
    :param is_report_loss_diff: Flag to report loss difference
    :param is_sample: Flag to sample randomly from the feasible area
    :return: The adversarial version of x (tensor)
    """
    model.eval()

    # Compute natural loss
    criterion = nn.CrossEntropyLoss(reduction='none')
    loss_natural = criterion(model(x), y.view(-1).long())

    # Initialize starting point
    x_next = x.clone()
    x_next = get_x0(x_next, initial_rounding_threshold, is_sample)

    # Multi-step PGD
    for t in range(k):
        # Forward pass
        x_var = x_next.clone().detach().requires_grad_(True)
        y_model = model(x_var)
        loss = criterion(y_model, y.view(-1).long())

        # Compute gradient
        grad_vars = torch.autograd.grad(loss.mean(), x_var)
        gradients = grad_vars[0].data

        #pos_insertion = (x_var <= 0.5) * 1 * insertion_array
        pos_insertion = (x_var <= 0.5) * 1
        grad4insertion = (gradients >= 0) * pos_insertion * gradients

        pos_removal = (x_var > 0.5) * 1 * removal_array
        grad4removal = (gradients < 0) * pos_removal * gradients

        gradients = grad4removal + grad4insertion
        #gradients =  grad4insertion

        # Norm
        if norm == 'linf':
            perturbation = torch.sign(gradients).float()

        elif norm == 'l2':
            l2norm = torch.linalg.norm(gradients, dim=-1, keepdim=True)
            perturbation = (gradients / l2norm).float()
            #perturbation = torch.minimum(torch.tensor(1., dtype=x.dtype, device=x.device), gradients / l2norm).float()
            #perturbation = torch.maximum(torch.tensor(-1., dtype=x.dtype, device=x.device), perturbation).float()
            #perturbation[torch.isnan(perturbation)] = 0.
            #perturbation[torch.isinf(perturbation)] = 1.

        elif norm == 'l1':
            # consider just features of a sample which are not updated yet(because our update is 0to1 or 1to0 not stepwise)
            un_mod = torch.abs(x_next - x_var) <= 1e-6
            gradients = gradients * un_mod

            val, _ = torch.topk(torch.abs(gradients), 1)
            perturbation = (torch.abs(gradients) >= val.expand_as(gradients)).float() * torch.sign(gradients).float()
            # stop perturbing the examples that are successful to evade the victim
            outputs = model(x_next)
            _, predicted = torch.topk(outputs, k=1)
            done = (predicted != y).squeeze()
            if torch.all(done):
                break
            perturbation[done] = 0.

        else:
            raise ValueError("Expect 'l1' or 'l2' or 'linf' norm.")

        # Update x_next
        x_next = torch.clamp(x_next + perturbation * step_length, min=0., max=1.)

    # Rounding step
    if random:
       round_threshold = torch.rand(x_next.size())
    x_next = round_x(x_next, round_threshold=round_threshold)

    # Feasible projection
    #x_next = or_float_tensors(x_next, x)

    # Compute adversarial loss
    loss_adv = criterion(model(x_next), y.view(-1).long()).data

    if is_report_loss_diff:
        outputs = model(x_next)
        _, predicted = torch.topk(outputs, k=1)
        done = (predicted != y).squeeze()
        print(f"PGD {norm}: Attack effectiveness {done.sum().item() / x.size()[0] * 100:.3f}%.")

    # Replace with natural if adversarial loss is higher
    replace_flag = (loss_adv < loss_natural).squeeze()
    x_next[replace_flag] = x[replace_flag]

    return x_next



```
def mimic_attack_effectiveness_optimized(test_loader, model, seed, trials=1000, device="cuda:0"):
  """
  Calculates the effectiveness of the mimic attack on the given model.

  Args:
      test_loader: A PyTorch dataloader containing the test data.
      model: The PyTorch model to be attacked.
      seed: The random seed for reproducibility.
      trials: The number of random samples to use from the benign class (default: 1000).
      device: The device to use for computations (default: "cuda:0" if available, otherwise "cpu").

  Returns:
      The effectiveness of the mimic attack as a percentage (float).
  """

  torch.manual_seed(seed)
  model.eval()

  # Initialize counters
  successful_attacks = 0
  total_malicious_samples = 0

  # Pre-select benign samples for efficiency
  benign_samples = []
  for x_batch, y_batch in test_loader:
    benign_samples.append(x_batch[y_batch.squeeze() == 0])

  ben_x = torch.cat(benign_samples, dim=0).to(device)

  # Clear unnecessary variables
  del benign_samples

  trials = min(trials, len(ben_x))


  for x_batch, y_batch in test_loader:
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
    malicious_samples = x_batch[y_batch.squeeze() == 1]

    if len(malicious_samples) > 0:
      # Expand dimensions for efficient broadcasting
      malicious_samples_expanded = malicious_samples.unsqueeze(1).expand(-1, trials, -1)

      # Generate random indices outside the loop
      seed += 1
      torch.manual_seed(seed)
      indices = torch.randperm(len(ben_x), device=device)[:trials]
      trial_vectors_expanded = ben_x[indices].unsqueeze(0)

      # Perform the mimic attack and update counters
      modified_x = torch.clamp(malicious_samples_expanded + trial_vectors_expanded, min=0., max=1.)
      _, done = get_loss(modified_x.view(-1, modified_x.shape[-1]), torch.ones(trials * malicious_samples.shape[0], 1, device=device), model)
      successful_attacks += (done.view(malicious_samples.shape[0], trials).sum(dim=1) > 0).sum().item()
      total_malicious_samples += malicious_samples.shape[0]

  # Calculate and print attack effectiveness
  attack_effectiveness = (successful_attacks / total_malicious_samples) * 100 if total_malicious_samples > 0 else 0
  print(f"Mimic attack effectiveness: {attack_effectiveness:.3f}%.")

  return attack_effectiveness  # Added return statement for clarity

```



In [ ]:
def mimicry(ben_x, malicious_samples, model, removal_array, trials=30, seed=230, is_report_loss_diff=False):
    """
    Perform a mimicry attack.

    Args:
    - ben_x (torch.Tensor): Benign samples tensor.
    - malicious_samples (torch.Tensor): Malicious samples tensor.
    - model (torch.nn.Module): PyTorch model used for the attack.
    - trials (int): Number of trials for the attack.
    - seed (int): Random seed for reproducibility.
    - is_report_loss_diff (bool): Flag to indicate whether to report attack effectiveness.

    Returns:
    - adv_x (torch.Tensor): Adversarial examples tensor.
    """
    model.eval()
    seed += 1
    torch.manual_seed(seed)

    # Ensure trials do not exceed the length of ben_x
    trials = min(trials, len(ben_x))

    # Get the number of malicious samples
    n_samples = len(malicious_samples)

    if n_samples > 0:
        # Expand dimensions for efficient broadcasting
        malicious_samples_expanded = malicious_samples.unsqueeze(1).expand(-1, trials, -1)
        fixed_malicious_samples_expanded = malicious_samples_expanded * (1 - removal_array)

        # Generate random indices for sampling from ben_x
        indices = torch.randperm(len(ben_x), device=ben_x.device)[:trials]
        trial_vectors_expanded = ben_x[indices].unsqueeze(0)

        # Perform the mimic attack
        pertbx = torch.clamp(fixed_malicious_samples_expanded + trial_vectors_expanded, min=0., max=1.).to(torch.float32)

        # Compute the loss and check if adversarial examples are successful
        loss, done = get_loss(pertbx.view(-1, pertbx.shape[-1]), torch.ones(n_samples * trials, 1, device=ben_x.device,dtype=torch.float32), model)

        # Add maximum loss to successful attacks to differentiate
        max_v = loss.max()
        loss[done] += max_v

        # Reshape the loss and done tensors
        loss = loss.view(n_samples, trials)
        done = done.view(n_samples, trials)

        # Report attack effectiveness if required
        if is_report_loss_diff:
            n_done = torch.any(done, dim=-1).sum()
            print(f"Mimicry*{trials}: Attack effectiveness {n_done / n_samples * 100:.3f}%.")

        # Get the index of the maximum loss for each sample
        _, indices = loss.max(dim=-1)
        adv_x = pertbx[torch.arange(n_samples), indices]

        del pertbx, loss, done, malicious_samples_expanded, trial_vectors_expanded

        return adv_x
    else:
        print("No malicious samples found.")
        return None

In [ ]:
def PGD_Max(x,y, model, attack_list = ['linf', 'l2', 'l1'],steps_max=5, is_sample = False, varepsilon = 1e-20):
    """
    PGD_Max adversarial attack.

    Args:
        x: Input data tensor (shape: [samples, features])
        y: Ground truth labels tensor (shape: [samples])
        model: Neural network model
        attack_list: List of norms for attacks (default: ['linf', 'l2', 'l1'])
        steps_max: Maximum number of steps (default: 5)
        is_sample: Flag to sample randomly from the feasible area (default: False)
        vaρεpsilon: Tolerance for stopping condition (default: 1e-20)

    Returns:
        Adversarial version of input data (tensor)
    """
    batch_size = x.shape[0]
    norm_params = {
        'l1': {'k': 50, 'step_length': 1.0},
        'l2': {'k': 200, 'step_length': 0.05},
        'linf': {'k': 500, 'step_length': 0.002}
    }

    model.eval()
    with torch.no_grad():
        loss, done = get_loss(x,y,model) #shape:[samples],[samples]

    pre_loss = loss
    n = x.shape[0]
    adv_x = x.detach().clone()
    stop_flag = torch.zeros(n, dtype=torch.bool) #[samples]

    for t in range(steps_max):
      num_remaining  = (~stop_flag).sum().item()
      print('number of remaining samples : ',num_remaining )
      if num_remaining  <= 0:
          break

      remaining_label = y[~stop_flag]
      pertbx = []

      for norm in attack_list:
          if norm in norm_params:
              params = norm_params[norm]
              perturbation = pgd(adv_x[~stop_flag], remaining_label, model, norm=norm, is_sample=is_sample, **params)
              print("the number of added features : ", (perturbation.sum() - adv_x[~stop_flag].sum())/len(adv_x[~stop_flag]))
              pertbx.append(perturbation)
          else:
              raise ValueError("Expected 'l1' or 'l2' or 'linf' norm.")


      # here pertbx.shape = a list of (number of attacks  ,(num_remaining ,features))
      pertbx = torch.vstack(pertbx)
      # here pertbx.shape = a tensor (num_remaining *number of attacks samples, features)

      with torch.no_grad():
        remaining_label_ext = torch.cat([remaining_label] * len(attack_list)) #(labels*number of attacks )
        loss, done = get_loss(pertbx, remaining_label_ext,model) #(labels*number of attacks )
        loss = loss.reshape(len(attack_list), num_remaining ).permute(1, 0) #(num_remaining ,number of attacks)
        done = done.reshape(len(attack_list), num_remaining ).permute(1, 0) #(num_remaining ,number of attacks)

        success_flag = torch.any(done, dim=-1) #(num_remaining )
        # for a sample, if there is at least one successful attack, we will select the one with maximum loss;
        # while if no attacks evade the victim successful, all perturbed examples are reminded for selection

        done[~torch.any(done, dim=-1)] = 1 #loss.shape=done.shape=(samples,number of attacks)
        loss = (loss * done.to(torch.float)) + torch.min(loss) * (~done).to(torch.float) #(num_remaining ,number of attacks)
        pertbx = pertbx.reshape(len(attack_list), num_remaining , x.shape[1]).permute([1, 0, 2])#(num_remaining ,attacks,features)
        _, indices = loss.max(dim=-1) # ans:(samples), max loss among attacks which worked, and max loss among all attacks for sample , none of them worked
        adv_x[~stop_flag] = pertbx[torch.arange(num_remaining ), indices]
        a_loss = loss[torch.arange(num_remaining ), indices]
        pre_stop_flag = stop_flag.clone()
        stop_flag[~stop_flag] = (torch.abs(pre_loss[~stop_flag] - a_loss) < varepsilon) | success_flag
        pre_loss[~pre_stop_flag] = a_loss

    return adv_x

In [ ]:
def PGD_Max2(x,y, model, attack_list = ['linf', 'l2', 'l1'],steps_max=5, is_sample = False, varepsilon = 1e-20, is_report_loss_diff = False):
    """
    PGD_Max adversarial attack.

    Args:
        x: Input data tensor (shape: [samples, features])
        y: Ground truth labels tensor (shape: [samples])
        model: Neural network model
        attack_list: List of norms for attacks (default: ['linf', 'l2', 'l1'])
        steps_max: Maximum number of steps (default: 5)
        is_sample: Flag to sample randomly from the feasible area (default: False)
        vaρεpsilon: Tolerance for stopping condition (default: 1e-20)

    Returns:
        Adversarial version of input data (tensor)
    """
    batch_size = x.shape[0]
    norm_params = {
        'l1': {'k': 50, 'step_length': 1.0, 'is_report_loss_diff':is_report_loss_diff},
        'l2': {'k': 200, 'step_length': 0.05, 'is_report_loss_diff':is_report_loss_diff},
        'linf': {'k': 500, 'step_length': 0.002, 'is_report_loss_diff':is_report_loss_diff}
    }

    model.eval()
    with torch.no_grad():
        loss, done = get_loss(x,y,model) #shape:[samples],[samples]

    pre_loss = loss
    n = x.shape[0]
    adv_x = x.detach().clone().to(x.device)
    stop_flag = torch.zeros(n, dtype=torch.bool).to(x.device) #[samples]

    for t in range(steps_max):
      num_remaining  = (~stop_flag).sum().item()
      #print('number of remaining samples : ',num_remaining )
      if num_remaining  <= 0:
          break

      remaining_label = y[~stop_flag]
      pertbx = []

      for norm in attack_list:
          if norm in norm_params:
              params = norm_params[norm]
              perturbation = pgd(adv_x[~stop_flag], remaining_label, model, norm=norm, is_sample=is_sample, **params)
              #print("the number of added features : ", (perturbation.sum() - adv_x[~stop_flag].sum())/len(adv_x[~stop_flag]))
              pertbx.append(perturbation)
          else:
              raise ValueError("Expected 'l1' or 'l2' or 'linf' norm.")


      # here pertbx.shape = a list of (number of attacks  ,(num_remaining ,features))
      pertbx = torch.vstack(pertbx)
      # here pertbx.shape = a tensor (num_remaining *number of attacks samples, features)

      with torch.no_grad():
        remaining_label_ext = torch.cat([remaining_label] * len(attack_list)) #(labels*number of attacks )
        loss, done = get_loss(pertbx, remaining_label_ext,model) #(labels*number of attacks )

        # for a sample, if there is at least one successful attack, we will select the one with maximum loss;
        # while if no attacks evade the victim successful, all perturbed examples are reminded for selection
        max_v = loss.amax()
        loss[done] += max_v

        loss = loss.reshape(len(attack_list), num_remaining ).permute(1, 0) #(num_remaining ,number of attacks)
        done = done.reshape(len(attack_list), num_remaining ).permute(1, 0) #(num_remaining ,number of attacks)

        success_flag = torch.any(done, dim=-1) #(num_remaining )

        pertbx = pertbx.reshape(len(attack_list), num_remaining , x.shape[1]).permute([1, 0, 2])#(num_remaining ,attacks,features)
        _, indices = loss.max(dim=-1) # ans:(samples), max loss among attacks which worked, and max loss among all attacks for sample , none of them worked
        adv_x[~stop_flag] = pertbx[torch.arange(num_remaining ), indices]
        a_loss = loss[torch.arange(num_remaining ), indices]
        pre_stop_flag = stop_flag.clone()
        stop_flag[~stop_flag] = (torch.abs(pre_loss[~stop_flag] - a_loss) < varepsilon) | success_flag
        pre_loss[~pre_stop_flag] = a_loss

    return adv_x

In [ ]:
def pgd_step(x, y, model, norm, k, step_length):
    """
    Projected Gradient Descent (PGD) adversarial attack for stepwise.
    :param y: Ground truth labels
    :param x: Feature vector
    :param model: Neural network model
    :param k: Number of steps
    :param step_length: Step size for each iteration
    :param norm: Norm used for perturbation ('linf' or 'l2')
    :return: The adversarial version of x (tensor)(not rounded)
    """
    model.eval()

    criterion = nn.CrossEntropyLoss(reduction='none')

    # Initialize starting point
    x_next = x.clone()

    # Multi-step PGD
    for t in range(k):
        # Forward pass
        x_var = x_next.clone().detach().requires_grad_(True)
        y_model = model(x_var)
        loss = criterion(y_model, y.view(-1).long())

        # Compute gradient
        grad_vars = torch.autograd.grad(loss.mean(), x_var)
        grad_data = grad_vars[0].data
        gradients = grad_data * (x < 0.5)

        # Norm
        if norm == 'linf':
            perturbation = torch.sign(gradients)
        elif norm == 'l2':
            l2norm = torch.linalg.norm(gradients, dim=-1, keepdim=True)
            perturbation = torch.minimum(torch.tensor(1., dtype=x.dtype, device=x.device), gradients / l2norm)
            perturbation[torch.isnan(perturbation)] = 0.
            perturbation[torch.isinf(perturbation)] = 1.
        elif norm == 'l1':
            #ignore the gradient of indice which is updated
            gradients = gradients * (x_next < 0.5)
            val, _ = torch.topk(gradients, 1)
            perturbation = torch.sign(gradients >= val.expand_as(gradients))
            # stop perturbing the examples that are successful to evade the victim
            outputs = model(x_next)
            _, predicted = torch.topk(outputs, k=1)
            done = (predicted != y).squeeze()
            if torch.all(done):
                break
            perturbation[done] = 0.
        else:
            raise ValueError("Expect 'l1' or 'l2' or 'linf' norm.")

        # Update x_next
        x_next = torch.clamp(x_next + perturbation * step_length, min=0., max=1.)

    #remove negative pertubations, we cant use OR function because we have values between (0,1) like 0.2 which we want to keep
    x_adv = (((x_next - x) >= 0) * x_next) + (((x_next - x) < 0) * x)
    return x_adv

In [ ]:
def StepwiseMax(
    x,
    label,
    model,
    attack_list=["linf", "l2", "l1"],
    step_lengths={"l1": 1.0, "l2": 0.05, "linf": 0.002},
    steps=100,
    step_check = 1,
    random_start=False,
    round_threshold=0.5,
    is_attacker=False,
    is_score_round = False
):
  """
    Stepwise max attack (mixture of pgd-l1, pgd-l2, pgd-linf).

    Args:
        x: Input data tensor (shape: [batch_size, feature_dim])
        label: Ground truth labels tensor (shape: [batch_size])
        model: Victim model
        attack_list: List of attack norms (default: ["linf", "l2", "l1"])
        step_lengths: Dictionary mapping norm to its step length (default: {"l1": 1.0, "l2": 0.05, "linf": 0.002})
        steps: Maximum number of iterations (default: 100)
        random_start: Use random starting point (default: False)
        round_threshold: Threshold for rounding real scalars (default: 0.5)
        is_attacker: Play the role of attacker (default: False)

    Returns:
        Adversarial examples tensor (same shape as x)
  """

  model.eval()

  step_check = 1
  if not is_attacker:
      step_checks = [1, 10, 25, 50]
      step_check = random.choice(step_checks)

  print(f"Step check: {step_check}")
  mini_steps = [step_check] * (steps // step_check)
  if steps % step_check != 0:
      mini_steps.append(steps % step_check)

  n, red_n = x.shape
  adv_x = x.detach().clone()
  pert_x_cont = None
  prev_done = None
  for i, mini_step in enumerate(mini_steps):
      with torch.no_grad():
          if i == 0 :
              adv_x = get_x0(adv_x, initial_rounding_threshold=round_threshold, is_sample=random_start)

          _, done = get_loss(adv_x, label, model)
      if torch.all(done):
          break
      if i == 0:
          adv_x[~done] = x[~done]  # recompute the perturbation under other penalty factors
          prev_done = done
      else:
          adv_x[~done] = pert_x_cont[~done[~prev_done]]
          prev_done = done

      num_sample_red = torch.sum(~done).item()
      pertbx = []
      for norm in attack_list:
          step_length = step_lengths.get(norm, step_lengths["l1"])
          perturbation = pgd_step(adv_x[~done], label[~done], model, norm, mini_step, step_length)
          #print("the number of added features(not rounded) ", norm,": ", perturbation.sum()/len(adv_x[~done]) - adv_x[~done].sum()/len(adv_x[~done]))
          #print("the number of added features(rounded) ", norm," : ",(round_x(perturbation, round_threshold).sum() - round_x(adv_x[~done], round_threshold).sum())/len(adv_x[~done]))
          pertbx.append(perturbation)
      with torch.no_grad():
          pertbx = torch.vstack(pertbx)

          n_attacks = len(attack_list)
          label_ext = torch.cat([label[~done]] * n_attacks)

          if (not is_attacker) and (not is_score_round):
              scores, _done = get_loss(pertbx, label_ext,model)
          else:
              scores, _done = get_loss(round_x(pertbx, round_threshold), label_ext, model)
          max_v = scores.amax() if scores.amax() > 0 else 0.
          scores[_done] += max_v

          pertbx = pertbx.reshape(n_attacks, num_sample_red, red_n).permute([1, 0, 2])
          scores = scores.reshape(n_attacks, num_sample_red).permute(1, 0)
          _2, s_idx = scores.max(dim=-1)
          pert_x_cont = pertbx[torch.arange(num_sample_red), s_idx]
          adv_x[~done] = pert_x_cont if not is_attacker else round_x(pert_x_cont, round_threshold)

  print(i)
  if is_attacker:
      adv_x = round_x(adv_x, round_threshold)
  with torch.no_grad():
      _, done = get_loss(adv_x, label, model)
      print(f"step-wise max: attack effectiveness {done.sum().item() / done.size()[0] * 100:.3f}%.")

  return adv_x

In [ ]:
def StepwiseMax2(
    x,
    label,
    model,
    attack_list=["linf", "l2", "l1"],
    step_lengths={"l1": 1.0, "l2": 0.05, "linf": 0.002},
    steps=100,
    step_check = 1,
    random_start=False,
    round_threshold=0.5,
    is_attacker=False,
    is_score_round = False
):
  """
    Stepwise max attack (mixture of pgd-l1, pgd-l2, pgd-linf).

    Args:
        x: Input data tensor (shape: [batch_size, feature_dim])
        label: Ground truth labels tensor (shape: [batch_size])
        model: Victim model
        attack_list: List of attack norms (default: ["linf", "l2", "l1"])
        step_lengths: Dictionary mapping norm to its step length (default: {"l1": 1.0, "l2": 0.05, "linf": 0.002})
        steps: Maximum number of iterations (default: 100)
        random_start: Use random starting point (default: False)
        round_threshold: Threshold for rounding real scalars (default: 0.5)
        is_attacker: Play the role of attacker (default: False)

    Returns:
        Adversarial examples tensor (same shape as x)
  """

  model.eval()

  step_check = 1
  if not is_attacker:
      step_checks = [1, 10, 25, 50]
      step_check = random.choice(step_checks)

  print(f"Step check: {step_check}")
  mini_steps = [step_check] * (steps // step_check)
  if steps % step_check != 0:
      mini_steps.append(steps % step_check)
  n, red_n = x.shape
  adv_x = x.detach().clone()
  pert_x_cont = None
  prev_done = None
  for i, mini_step in enumerate(mini_steps):
      with torch.no_grad():
          if i == 0 :
              adv_x = get_x0(adv_x, initial_rounding_threshold=round_threshold, is_sample=random_start)

          if is_attacker:
            _, done = get_loss(round_x(adv_x, round_threshold), label, model)
          else :
            _, done = get_loss(adv_x, label, model)
      if torch.all(done):
          break
      if i == 0:
          adv_x[~done] = x[~done]  # recompute the perturbation under other penalty factors
          prev_done = done
      else:
          adv_x[~done] = pert_x_cont[~done[~prev_done]]
          prev_done = done

      print('len(adv_x[~done]) : ',len(adv_x[~done]))
      num_sample_red = torch.sum(~done).item()
      pertbx = []
      for norm in attack_list:
          step_length = step_lengths.get(norm, step_lengths["l1"])
          perturbation = pgd_step(adv_x[~done], label[~done], model, norm, mini_step, step_length)
          #print("the number of added features(not rounded) ", norm,": ", perturbation.sum()/len(adv_x[~done]) - adv_x[~done].sum()/len(adv_x[~done]))
          #print("the number of added features(rounded) ", norm," : ",(round_x(perturbation, round_threshold).sum() - round_x(adv_x[~done], round_threshold).sum())/len(adv_x[~done]))
          pertbx.append(perturbation)
      with torch.no_grad():
          pertbx = torch.vstack(pertbx)

          n_attacks = len(attack_list)
          label_ext = torch.cat([label[~done]] * n_attacks)

          if (is_score_round):
              scores, _done = get_loss(round_x(pertbx, round_threshold), label_ext, model)
          elif is_attacker:
              scores, _ = get_loss(pertbx, label_ext,model)
              _, _done = get_loss(round_x(pertbx, round_threshold), label_ext, model)
          else:
              scores, _done = get_loss(pertbx, label_ext,model)

          max_v = scores.amax() if scores.amax() > 0 else 0.
          scores[_done] += max_v

          pertbx = pertbx.reshape(n_attacks, num_sample_red, red_n).permute([1, 0, 2])
          scores = scores.reshape(n_attacks, num_sample_red).permute(1, 0)
          _2, s_idx = scores.max(dim=-1)
          pert_x_cont = pertbx[torch.arange(num_sample_red), s_idx]
          adv_x[~done] = pert_x_cont

  print(i)
  if is_attacker:
      adv_x = round_x(adv_x, round_threshold)
  with torch.no_grad():
      _, done = get_loss(adv_x, label, model)
      print(f"step-wise max: attack effectiveness {done.sum().item() / done.size()[0] * 100:.3f}%.")

  return adv_x

In [ ]:
def pgd_one_step(x, y, model, step_lengths):
    """
    Projected Gradient Descent (PGD) adversarial attack for stepwise.
    :param y: Ground truth labels
    :param x: Feature vector
    :param model: Neural network model
    :param k: Number of steps
    :param step_length: Step size for each iteration
    :param norm: Norm used for perturbation ('linf' or 'l2')
    :return: The adversarial version of x (tensor)(not rounded)
    """
    model.eval()

    criterion = nn.CrossEntropyLoss(reduction='none')

    # Initialize starting point
    x_next = x.clone()

    # one-step PGD

    # Forward pass
    x_var = x_next.clone().detach().requires_grad_(True)
    y_model = model(x_var)
    loss = criterion(y_model, y.view(-1).long())

    # Compute gradient
    grad_vars = torch.autograd.grad(loss.mean(), x_var)
    grad_data = grad_vars[0].data
    gradients = grad_data * (x < 0.5)


    # Norms
    pertbx = []
    # norm = linf
    step_length = step_lengths.get("linf", step_lengths["l1"])
    perturbation_linf = torch.sign(gradients)
    x_next_linf = torch.clamp(x_next + perturbation_linf * step_length, min=0., max=1.)
    #remove negative pertubations, we cant use OR function because we have values between (0,1) like 0.2 which we want to keep
    x_adv_linf = (((x_next_linf - x) >= 0) * x_next_linf) + (((x_next_linf - x) < 0) * x)
    pertbx.append(x_adv_linf)
    # norm = l2
    step_length = step_lengths.get("l2", step_lengths["l1"])
    l2norm = torch.linalg.norm(gradients, dim=-1, keepdim=True)
    perturbation_l2 = torch.minimum(torch.tensor(1., dtype=x.dtype, device=x.device), gradients / l2norm)
    perturbation_l2[torch.isnan(perturbation_l2)] = 0.
    perturbation_l2[torch.isinf(perturbation_l2)] = 1.
    x_next_l2 = torch.clamp(x_next + perturbation_l2 * step_length, min=0., max=1.)
    x_adv_l2 = (((x_next_l2 - x) >= 0) * x_next_l2) + (((x_next_l2 - x) < 0) * x)
    pertbx.append(x_adv_l2)
    # norm = l1
    step_length = step_lengths.get("l1", step_lengths["l1"])
    #ignore the gradient of indice which is updated
    gradients = gradients * (x_next < 0.5)
    val, _ = torch.topk(gradients, 1)
    perturbation_l1 = torch.sign(gradients >= val.expand_as(gradients))
    x_next_l1 = torch.clamp(x_next + perturbation_l1 * step_length, min=0., max=1.)
    x_adv_l1 = (((x_next_l1 - x) >= 0) * x_next_l1) + (((x_next_l1 - x) < 0) * x)
    pertbx.append(x_adv_l1)

    return pertbx

In [ ]:
def StepwiseMax_onestep(
    x,
    label,
    model,
    attack_list=["linf", "l2", "l1"],
    step_lengths={"l1": 1.0, "l2": 0.05, "linf": 0.002},
    steps=100,
    random_start=False,
    round_threshold=0.5,
):
  """
    Stepwise max attack (mixture of pgd-l1, pgd-l2, pgd-linf).

    Args:
        x: Input data tensor (shape: [batch_size, feature_dim])
        label: Ground truth labels tensor (shape: [batch_size])
        model: Victim model
        attack_list: List of attack norms (default: ["linf", "l2", "l1"])
        step_lengths: Dictionary mapping norm to its step length (default: {"l1": 1.0, "l2": 0.05, "linf": 0.002})
        steps: Maximum number of iterations (default: 100)
        random_start: Use random starting point (default: False)
        round_threshold: Threshold for rounding real scalars (default: 0.5)

    Returns:
        Adversarial examples tensor (same shape as x)
  """

  model.eval()

  n, red_n = x.shape
  adv_x = x.detach().clone()
  pert_x_cont = None
  prev_done = None
  for step in range(steps):
      with torch.no_grad():
          if step == 0 :
              adv_x = get_x0(adv_x, initial_rounding_threshold=round_threshold, is_sample=random_start)

          _, done = get_loss(round_x(adv_x, round_threshold), label, model)

      if torch.all(done):
          break
      if step == 0:
          adv_x[~done] = x[~done]  # recompute the perturbation under other penalty factors
          prev_done = done
      else:
          adv_x[~done] = pert_x_cont[~done[~prev_done]]
          prev_done = done

      #print('len(adv_x[~done]) : ',len(adv_x[~done]))
      num_sample_red = torch.sum(~done).item()

      pertbx = pgd_one_step(adv_x[~done], label[~done], model, step_lengths)

      with torch.no_grad():
          pertbx = torch.vstack(pertbx)

          n_attacks = len(attack_list)
          label_ext = torch.cat([label[~done]] * n_attacks)

          scores, _ = get_loss(pertbx, label_ext,model)
          _, _done = get_loss(round_x(pertbx, round_threshold), label_ext, model)

          max_v = scores.amax() if scores.amax() > 0 else 0.
          scores[_done] += max_v

          pertbx = pertbx.reshape(n_attacks, num_sample_red, red_n).permute([1, 0, 2])
          scores = scores.reshape(n_attacks, num_sample_red).permute(1, 0)
          _2, s_idx = scores.max(dim=-1)
          pert_x_cont = pertbx[torch.arange(num_sample_red), s_idx]
          adv_x[~done] = pert_x_cont

  #print(step)
  adv_x = round_x(adv_x, round_threshold)
  with torch.no_grad():
      _, done = get_loss(adv_x, label, model)
      print(f"step-wise max: attack effectiveness {done.sum().item() / done.size()[0] * 100:.3f}%.")

  return adv_x

In [ ]:
def around_x(x, std_deviation=0.1, random_start=False):
    """
    Helper function to randomly initialize the inner maximizer algorithm.
    Randomizes the input tensor while preserving its functionality.
    :param x: input tensor
    :param std_deviation: std_deviation for domain
    :param random_start: flag to sample randomly from feasible area
    :return: randomly sampled feasible version of x
    """
    if random_start:

        # Generate random tensor from a Gaussian distribution centered around zero
        random_tensor = abs(torch.randn(x.size()))

        # Scale the values to control the spread of the distribution
        random_tensor *= std_deviation

        return torch.clamp(x + random_tensor, min=0., max=1.)
    else:
        return x


In [ ]:
def pgd_one_step2(x, y, model, step_lengths,x_initial):
    """
    Projected Gradient Descent (PGD) adversarial attack for stepwise.
    :param y: Ground truth labels
    :param x: Feature vector
    :param model: Neural network model
    :param k: Number of steps
    :param step_length: Step size for each iteration
    :param norm: Norm used for perturbation ('linf' or 'l2')
    :return: The adversarial version of x (tensor)(not rounded)
    """
    model.eval()

    criterion = nn.CrossEntropyLoss(reduction='none')

    # Initialize starting point
    x_next = x.clone()

    # one-step PGD

    # Forward pass
    x_var = x_next.clone().detach().requires_grad_(True)
    y_model = model(x_var)
    loss = criterion(y_model, y.view(-1).long())

    # Compute gradient
    grad_vars = torch.autograd.grad(loss.mean(), x_var)
    grad_data = grad_vars[0].data
    gradients = grad_data * (x_initial < 0.5)


    # Norms
    pertbx = []
    # norm = linf
    step_length = step_lengths.get("linf", step_lengths["l1"])
    perturbation_linf = torch.sign(gradients)
    x_next_linf = torch.clamp(x_next + perturbation_linf * step_length, min=0., max=1.)
    #remove negative pertubations, we cant use OR function because we have values between (0,1) like 0.2 which we want to keep
    x_adv_linf = (((x_next_linf - x_initial) >= 0) * x_next_linf) + (((x_next_linf - x_initial) < 0) * x_initial)
    pertbx.append(x_adv_linf)
    # norm = l2
    step_length = step_lengths.get("l2", step_lengths["l1"])
    l2norm = torch.linalg.norm(gradients, dim=-1, keepdim=True)
    perturbation_l2 = torch.minimum(torch.tensor(1., dtype=x.dtype, device=x.device), gradients / l2norm)
    perturbation_l2[torch.isnan(perturbation_l2)] = 0.
    perturbation_l2[torch.isinf(perturbation_l2)] = 1.
    x_next_l2 = torch.clamp(x_next + perturbation_l2 * step_length, min=0., max=1.)
    x_adv_l2 = (((x_next_l2 - x_initial) >= 0) * x_next_l2) + (((x_next_l2 - x_initial) < 0) * x_initial)
    pertbx.append(x_adv_l2)
    # norm = l1
    step_length = step_lengths.get("l1", step_lengths["l1"])
    #ignore the gradient of indice which is updated
    gradients_l1 = gradients * (x_next < 0.5)
    val, _ = torch.topk(gradients_l1, 1)

    perturbation_l1 = torch.sign(gradients >= val.expand_as(gradients)) * (val > 1e-10)

    x_next_l1 = torch.clamp(x_next + perturbation_l1 * step_length, min=0., max=1.)
    x_adv_l1 = (((x_next_l1 - x_initial) >= 0) * x_next_l1) + (((x_next_l1 - x_initial) < 0) * x_initial)
    pertbx.append(x_adv_l1)

    return pertbx

In [ ]:
def StepwiseMax_onestep2(
    x,
    label,
    model,
    attack_list=["linf", "l2", "l1"],
    step_lengths={"l1": 1.0, "l2": 0.05, "linf": 0.002},
    steps=100,
    random_start=False,
    round_threshold=0.5,
):
  """
    Stepwise max attack (mixture of pgd-l1, pgd-l2, pgd-linf).

    Args:
        x: Input data tensor (shape: [batch_size, feature_dim])
        label: Ground truth labels tensor (shape: [batch_size])
        model: Victim model
        attack_list: List of attack norms (default: ["linf", "l2", "l1"])
        step_lengths: Dictionary mapping norm to its step length (default: {"l1": 1.0, "l2": 0.05, "linf": 0.002})
        steps: Maximum number of iterations (default: 100)
        random_start: Use random starting point (default: False)
        round_threshold: Threshold for rounding real scalars (default: 0.5)

    Returns:
        Adversarial examples tensor (same shape as x)
  """

  model.eval()

  n, red_n = x.shape
  adv_x = x.detach().clone()
  pert_x_cont = None
  prev_done = None
  for step in range(steps):
      with torch.no_grad():
          if step == 0 :
              adv_x = around_x(adv_x, std_deviation=0.1, random_start=random_start)

          _, done = get_loss(round_x(adv_x, round_threshold), label, model)

      if torch.all(done):
          break
      if step == 0:
          adv_x[~done] = x[~done]  # recompute the perturbation under other penalty factors
          prev_done = done
      else:
          adv_x[~done] = pert_x_cont[~done[~prev_done]]
          prev_done = done

      #print('remaining samples : ',len(adv_x[~done]))
      num_sample_red = torch.sum(~done).item()

      pertbx = pgd_one_step2(adv_x[~done], label[~done], model, step_lengths,x[~done])

      with torch.no_grad():
          pertbx = torch.vstack(pertbx)

          n_attacks = len(attack_list)
          label_ext = torch.cat([label[~done]] * n_attacks)

          scores, _ = get_loss(pertbx, label_ext,model)
          _, _done = get_loss(round_x(pertbx, round_threshold), label_ext, model)

          max_v = scores.amax() if scores.amax() > 0 else 0.
          scores[_done] += max_v

          pertbx = pertbx.reshape(n_attacks, num_sample_red, red_n).permute([1, 0, 2])
          scores = scores.reshape(n_attacks, num_sample_red).permute(1, 0)
          _2, s_idx = scores.max(dim=-1)
          print('best attack : ',s_idx)
          pert_x_cont = pertbx[torch.arange(num_sample_red), s_idx]
          adv_x[~done] = pert_x_cont

  #print(step)
  adv_x = round_x(adv_x, round_threshold)
  with torch.no_grad():
      _, done = get_loss(adv_x, label, model)
      print(f"step-wise max: attack effectiveness {done.sum().item() / done.size()[0] * 100:.3f}%.")

  return adv_x